# Evaulate Lonformer Model on Politifact
### 1 Dec 2020

## I. Evaluation Parameters

In [11]:
# Set Evaulation Parameters
MODEL_NAME = "allenai/longformer-base-4096"
MODEL_FILE ="saved_model_epoch10_20201130_0855.tar"
TEST_DATASET = "test_data_29Nov_longformer_pf.pickle"
EVAL_BATCH_SIZE = 8
DESCRIPTION = "PolitiFact only model on Longformer"

## II. Setup and Load Model

In [2]:
import pickle
import sys

import json
import pandas as pd
import pickle
import sklearn.metrics
import tqdm
import torch
import transformers as hft

sys.path.insert(0, "/home/jupyter")
import util.log
import util.data

In [4]:
# Load evaluation data
with open(TEST_DATASET, "rb") as vfile:
    test_dataset = pickle.load(vfile)

In [5]:
# Load saved model
checkpoint = torch.load(MODEL_FILE)
model = (hft.LongformerForSequenceClassification
         .from_pretrained(MODEL_NAME))
model.load_state_dict(checkpoint["model"])

Some weights of the model checkpoint at allenai/longformer-base-4096 were not used when initializing LongformerForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing LongformerForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LongformerForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of LongformerForSequenceClassification were not initialized from the model checkpoint at allenai/longformer-base-4096 and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', '

<All keys matched successfully>

In [6]:
# Move model to GPU if available
device = (torch.device('cuda') if torch.cuda.is_available()
          else torch.device('cpu'))
model.to(device)
model.eval();
device

device(type='cuda')

## III. Evaluation Loop and Results Dataframe

In [8]:
# Evaulate model
eval_loader = torch.utils.data.DataLoader(test_dataset,
                                batch_size=EVAL_BATCH_SIZE,
                                shuffle=False)
labels = []
preds = []
logits = []
probs = []
sources = []
article_tokens = []
file_names = []

softmax = torch.nn.Softmax(dim=1)

with torch.no_grad():
    for art_num, article in tqdm.tqdm(enumerate(eval_loader)):
        input_ids = article["input_ids"].to(device)
        attent_mask = article["attention_mask"].to(device)
        output = model(input_ids,
                       attention_mask=attent_mask,
                       output_hidden_states=False,
                       output_attentions=False)
        prob = softmax(output[0].detach().cpu()).numpy()
        logit = output[0].detach().cpu().numpy()
        pred = [0 if lgt[0] > lgt[1] else 1 for lgt in logit]
        label = article["labels"].numpy()
        labels.extend(label)
        preds.extend(pred)
        logits.extend(logit)
        probs.extend(prob)
        article_tokens.extend(article["article_tokens"].numpy())
        sources.extend(article["sources"])
        file_names.extend(article["file_names"])

16it [00:36,  2.30s/it]


In [9]:
# Label of 0 is real, 1 is fake
eval_results = pd.DataFrame({"Article": list(range(len(labels))),
                             "Source": sources,
                             "Token_Length": article_tokens,
                             "Predictions": preds,
                             "Labels": labels,
                             "Logit_real(0)": [x[0] for x in logits],
                             "Logit_fake(1)": [x[1] for x in logits],
                             "Prob_real(0)": [x[0] for x in probs],
                             "Prob_real(1)": [x[1] for x in probs],
                             "File_Name": file_names})

eval_results

,Article,Source,Token_Length,Predictions,Labels,Logit_real(0),Logit_fake(1),Prob_real(0),Prob_real(1),File_Name
0,0,politifact,7017,0,0,4.091337,-3.210758,0.999326,0.000674,politifact3584
1,1,politifact,1861,1,1,-3.084408,2.261991,0.004743,0.995257,politifact13565
2,2,politifact,331,1,1,-3.674033,2.612231,0.001858,0.998142,politifact14211
3,3,politifact,1547,0,0,4.103150,-3.240576,0.999354,0.000646,politifact1361
4,4,politifact,11148,0,0,4.102195,-3.229703,0.999346,0.000654,politifact3281
...,...,...,...,...,...,...,...,...,...,...
119,119,politifact,1377,1,1,-3.741538,2.667076,0.001645,0.998355,politifact14777
120,120,politifact,534,0,0,4.018033,-2.997874,0.999103,0.000897,politifact1500
121,121,politifact,21669,0,0,4.042229,-3.189760,0.999277,0.000723,politifact13260
122,122,politifact,1029,1,1,-3.711784,2.656457,0.001712,0.998288,politifact15544


In [10]:
# Save evaluation data to csv file
eval_results.to_csv("test_resuts.csv")

## IV. Overall Metrics

In [12]:
# Overall metrics
precision, recall, f1, _ = sklearn.metrics.precision_recall_fscore_support(
    eval_results.Labels, eval_results.Predictions, average="binary")
accuracy = sklearn.metrics.accuracy_score(eval_results.Labels, eval_results.Predictions)

metrics = {"description": DESCRIPTION,
           "eval_notes": "Evaluating PolitiFact data",
           "model": MODEL_FILE,
           "epochs": checkpoint["epoch"],
           "batch_size": EVAL_BATCH_SIZE,
           "Date_evaluated": "1DEC2020",
           "test_data": "test_resuts.csv",
           "eval_data": TEST_DATASET,
           "accuracy": accuracy,
           "precision": precision,
           "recall": recall,
           "f1": f1}
metrics

{'description': 'PolitiFact only model on Longformer',
 'eval_notes': 'Evaluating PolitiFact data',
 'model': 'saved_model_epoch10_20201130_0855.tar',
 'epochs': 10,
 'batch_size': 8,
 'Date_evaluated': '1DEC2020',
 'test_data': 'test_resuts.csv',
 'eval_data': 'test_data_29Nov_longformer_pf.pickle',
 'accuracy': 0.9516129032258065,
 'precision': 0.9152542372881356,
 'recall': 0.9818181818181818,
 'f1': 0.9473684210526316}

In [13]:
# Save metrics to JSON text file.
with open("test_metrics.json", "wt") as jfile:
    json.dump(metrics, jfile)

In [14]:
cm = pd.DataFrame(sklearn.metrics.confusion_matrix(eval_results.Labels, eval_results.Predictions))
label_titles = {0: "real", 1: "fake"}
cm.rename(index=label_titles, columns=label_titles, inplace=True)
cm

,real,fake
real,64,5
fake,1,54
